In [179]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [202]:
# Import data
data_path = '/Users/lukecullen/PycharmProjects/petrochemical-data/data/'
estimated_file = data_path+'combined/240325_All/facilityEmissions_all.parquet'

start_year = 2020
end_year = 2050
CCS_gas = 'Carbon dioxide'
gas_nx = 'Nitric oxide' # Actually nitrous oxide, this is to compensate for previous misnamings
CCS_ratio = 1
efficiency = 0.9
summary_gases = ['CO2e_20a', 'CO2e_100a']
process = 'Direct Utilities'

In [203]:
data = pd.read_parquet(estimated_file)
dp_emissions = data[data['Type']==process]
#deu_emissions = data[data['Type']=='Direct Utilities']

In [204]:
a = 1
b = 0.35
c = 1
M = 15
y = range(0, end_year-start_year+1)

penetration = a / (1+np.exp(-b*(np.array(y)-M)))**a

years = [str(i) for i in range(start_year, end_year+1)]
years_sigma = [year+'_sigma' for year in years]

df = dp_emissions.copy()

In [205]:
def convert_ammonia_to_plants(df):
    
    # Convert ammonia countries to plants
    years = [str(i) for i in range(1978, 2051)]
    years_sigma = [year+'_sigma' for year in years]
    
    ammonia_df = pd.DataFrame()
    num_countries_amm = round(127/2)
    
    for num, i in enumerate(df[df['PRODUCT'] == 'AMMONIA'].sort_values(by='2050', ascending=False).iloc()):
        subtraction = round((num_countries_amm-num/2))
        # Make df with line i repeated 100 times and with values equal to 1/100 of the original
        i[years] = i[years].values/subtraction
        i[years_sigma] = i[years_sigma].values/subtraction
        # Create a df with the line repeated 100 times
        df_repeated = pd.DataFrame(np.repeat(i.values.reshape(1, -1), subtraction, axis=0), columns=i.index)
        ammonia_df = pd.concat((ammonia_df, df_repeated))
    
    df = pd.concat((df[df['PRODUCT'] != 'AMMONIA'], ammonia_df))
    return df

In [206]:
def get_facility_reduction(df, CCS_gas, CCS_ratio, penetration):
    gas_emissions = df[df['Gas']==CCS_gas]
        
    gas_emissions = convert_ammonia_to_plants(gas_emissions)
    
    sorted_emissions = gas_emissions.sort_values(by='2050', ascending=False)
    contributing_facils = len(sorted_emissions[sorted_emissions['2050'] > 0])
    
    years = [str(i) for i in range(start_year, end_year+1)]
    years_sigma = [year+'_sigma' for year in years]
    
    multiplication_vec = np.ones(len(sorted_emissions))
    resulting_emissions = sorted_emissions.copy()
    
    for num, year in enumerate(years):
        year_vec = multiplication_vec.copy()
        facil_numbers = int(np.floor(contributing_facils*CCS_ratio*penetration[num]))
        year_vec[:facil_numbers] = 1-efficiency
        resulting_emissions[year] = resulting_emissions[year]*year_vec
        resulting_emissions[year+'_sigma'] = resulting_emissions[year+'_sigma']*year_vec
    

    # Regroup ammonia
    resulting_emissions = pd.concat((resulting_emissions[resulting_emissions['PRODUCT'] != 'AMMONIA'], resulting_emissions[resulting_emissions['PRODUCT'] == 'AMMONIA'].fillna(0).groupby(list(gas_emissions.columns[:15])+list(gas_emissions.columns[-3:])).sum().reset_index()))

    sorted_emissions = pd.concat((sorted_emissions[sorted_emissions['PRODUCT'] != 'AMMONIA'], sorted_emissions[sorted_emissions['PRODUCT'] == 'AMMONIA'].fillna(0).groupby(list(gas_emissions.columns[:15])+list(gas_emissions.columns[-3:])).sum().reset_index()))
    
    resulting_emissions = resulting_emissions.sort_values(by=list(gas_emissions.columns[:15])+list(gas_emissions.columns[-3:]))
    sorted_emissions = sorted_emissions.sort_values(by=list(gas_emissions.columns[:15])+list(gas_emissions.columns[-3:]))
    
    gas_reduction = sorted_emissions[years].values - resulting_emissions[years].values
    sigma_reduction = sorted_emissions[years_sigma].values - resulting_emissions[years_sigma].values

    return gas_reduction, sigma_reduction, resulting_emissions 

In [207]:
# def get_gas_reduction(df, CCS_gas, CCS_ratio, penetration, years, years_sigma):
#     gas_emissions = df[df['Gas']==CCS_gas]
#     new_emissions = gas_emissions[years]*(1-penetration*CCS_ratio)
#     new_sigmas = gas_emissions[years_sigma]*(1-penetration*CCS_ratio)
#     
#     gas_reduction = gas_emissions[years].values - new_emissions.values
#     sigma_reduction = gas_emissions[years_sigma].values - new_sigmas.values
#     gas_emissions[years] = new_emissions
#     gas_emissions[years_sigma] = new_sigmas
#     
#     return gas_reduction, sigma_reduction, gas_emissions


def get_summary_emissions(df, summary_gas, gas_reduction, sigma_reduction, years, years_sigma, gas_ratio=1):
    """Gas ratio is the ratio of the gas greenhouse effect relative to CO2 for the summary period (20yr or 100yr) desired"""
    summary_emissions = df[df['Gas']==summary_gas]
    summary_emissions[years] = summary_emissions[years].values - gas_reduction*gas_ratio
    summary_emissions[years_sigma] = summary_emissions[years_sigma].values - sigma_reduction*gas_ratio
    return summary_emissions

df = df.sort_values(by=list(df.columns[:15])+list(df.columns[-3:]))

gas_reduction, sigma_reduction, gas_emissions = get_facility_reduction(df, CCS_gas, CCS_ratio, penetration)

# For first gas
ccs_emissions = df[[i not in [CCS_gas]+summary_gases for i in df['Gas']]]

for summary_gas in summary_gases:
    summary_emissions = get_summary_emissions(df, summary_gas, gas_reduction, sigma_reduction, years, years_sigma)
    ccs_emissions = pd.concat((ccs_emissions, summary_emissions))
    
ccs_emissions = pd.concat((ccs_emissions, gas_emissions))

/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h0000gn/T/ipykernel_15213/1103265125.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[years] = i[years].values/subtraction
/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h0000gn/T/ipykernel_15213/1103265125.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[years] = i[years].values/subtraction
/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h0000gn/T/ipykernel_15213/1103265125.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [208]:
# For second gas
gas_reduction_nx, sigma_reduction_nx, gas_emissions_nx = get_facility_reduction(ccs_emissions, gas_nx, CCS_ratio, penetration)

ccs_emissions_nx = ccs_emissions[[i not in [gas_nx]+summary_gases for i in ccs_emissions['Gas']]]

for summary_gas in summary_gases:
    summary_emissions = get_summary_emissions(ccs_emissions, summary_gas, gas_reduction_nx, sigma_reduction_nx, years, years_sigma, gas_ratio=281)
    ccs_emissions_nx = pd.concat((ccs_emissions_nx, summary_emissions))
    
ccs_emissions_nx = pd.concat((ccs_emissions_nx, gas_emissions_nx))

/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h0000gn/T/ipykernel_15213/1103265125.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[years] = i[years].values/subtraction
/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h0000gn/T/ipykernel_15213/1103265125.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[years] = i[years].values/subtraction
/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h0000gn/T/ipykernel_15213/1103265125.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [209]:
#ccs_emissions.to_csv(data_path+'scenarios/dp_ccs_emissions.csv')
ccs_emissions_nx.to_csv(data_path+'scenarios/dp_ccs_emissions_facilities_'+process+'_'+str(CCS_ratio*efficiency)[2:]+'.csv')